In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df2_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AS_segment2_workflowS4_has_duration.csv",engine='python')

In [3]:
df2_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,722111|418134|648088,Release Offer,Release Offer,8/12/2018 14:00:19,67 days 0 hours 15 minutes 20 seconds,5789720.0


In [4]:
df2_WS4T['WRKFLW_TSK_MAP_GROUPED'].nunique()

35

In [5]:
df2_WS4T['WRKFLW_TSK_MAP_DETAILED'].nunique()

43

In [6]:
wd_count = df2_WS4T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Release Offer', 1109), ('Accept', 799), ('OC - Satisfied - Evidence of course completion', 227), ('Manage Acceptance', 114), ('Defer', 83), ('OC - Created - Evidence of course completion', 40), ('Decline', 35), ('Assessment Decision - Qualified', 33), ('AR - Satisfied - Academic results for prior tertiary studies', 18), ('OC - Satisfied - English language requirements', 17), ('Assessment Decision - No Decision', 13), ('AR - Satisfied - Further information (1)', 10), ('AR - Satisfied - Credit for previous study', 8), ('AR - Satisfied - English language proficiency', 8), ('OC - Not Satisfied - Evidence of course completion', 7), ('OC - Requested - Evidence of course completion', 7), ('Release Assessment Hold', 6), ('Withdraw', 5), ('Put Assessment On-Hold', 4), ('AR - Satisfied - Graduate Access Melbourne', 3), ('End My Assessment', 3), ('OC - Created - English language requirements', 3), ('OC - Reviewed - Evidence of course completion', 3), ('AR - Reviewed - Credit for previous study

In [7]:
list = df2_WS4T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df2_WS4T[df2_WS4T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('Release Assessment Hold', 5331729.0), ('OC - Created - Evidence of course completion', 3843409.0), ('OC - Reviewed - Evidence of course completion', 3741198.0), ('AR - Satisfied - Official syllabus or subject descriptions (in English)', 2972607.0), ('Release Offer', 2818026.0), ('AR - Satisfied - Further information � Preference order required as submitted multiple 1st prefer applications', 2406651.0), ('OC - Created - CHESICC transcript authentication for international students', 2164642.0), ('Assessment Decision - Qualified', 1860143.0), ('AR - Satisfied - Academic results for prior tertiary studies', 1751416.0), ('AR - Satisfied - Credit for previous study', 1658817.0), ('AR - Reviewed - Credit for previous study', 1335121.0), ('AR - Reviewed - Graduate Access Melbourne', 1305685.0), ('AR - Satisfied - Graduate Access Melbourne', 1260915.0), ('OC - Created - English language requirements', 1160933.0), ('OC - Satisfied - English language requirements', 1103938.0), ('AR - Satisfied

In [8]:
numOfWorkflowProcesses = df2_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic

count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df2_WS4T[df2_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df2_WS4T[df2_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df2_WS4T[df2_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df2_WS4T[df2_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df2_WS4T[df2_WS4T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)


In [16]:
s2_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s2_t4['Num of Occurrences per step(descending)'] = list_step_count_per_key
s2_t4['Average time spends per step(descending)'] = list_step_ave_time_per_key
s2_t4['average time for workflow segment(seconds)'] = list_avgtime_per_key
s2_t4['sum time for workflow segment(seconds)'] = list_sumtime_row
s2_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,1,126,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,"[(Release Offer, 126)]",[],0,0.000000e+00
1,2,601,31 days 9 hours 25 minutes 56 seconds,15.0 days 7.0 hours 9.0 minutes 42.0 seconds,45.01%,"[(Release Offer, 602), (Accept, 482), (Defer, ...","[(Release Offer, 31.0 days 8.0 hours 10.0 minu...",2712356,1.630126e+09
2,3,301,63 days 12 hours 49 minutes 11 seconds,49.0 days 14.0 hours 3.0 minutes 4.0 seconds,30.56%,"[(Release Offer, 302), (Accept, 257), (OC - Sa...","[(AR - Satisfied - further information, 8.0 da...",5489351,1.652295e+09
3,4,53,57 days 23 hours 57 minutes 14 seconds,49.0 days 8.0 hours 13.0 minutes 48.0 seconds,21.7%,"[(Release Offer, 54), (Accept, 42), (OC - Sati...","[(OC - Reviewed - Other, 43.0 days 8.0 hours 4...",5011034,2.655848e+08
4,5,14,75 days 10 hours 12 minutes 26 seconds,46.0 days 20.0 hours 47.0 minutes 21.0 seconds,15.71%,"[(Release Offer, 15), (Accept, 10), (OC - Sati...","[(Assessment Decision - Qualified, 86.0 days 1...",6516746,9.123445e+07
5,6,6,36 days 18 hours 2 minutes 35 seconds,24.0 days 16.0 hours 41.0 minutes 58.5 seconds,13.89%,"[(Release Offer, 6), (OC - Satisfied - Other, ...","[(AR - Satisfied - advanced standing, 4.0 days...",3175355,1.905213e+07
6,7,2,108 days 9 hours 26 minutes 9 seconds,108.0 days 9.0 hours 26.0 minutes 9.0 seconds,14.29%,"[(Assessment Decision - Qualified, 3), (Releas...","[(OC - Reviewed - Other, 43.0 days 4.0 hours 1...",9365169,1.873034e+07
7,8,1,77 days 0 hours 46 minutes 4 seconds,77.0 days 0.0 hours 46.0 minutes 4.0 seconds,0.0%,"[(Assessment Decision - Not Qualified, 2), (OC...","[(Assessment Decision - No Decision, 6.0 days ...",6655564,6.655564e+06
8,11,1,38 days 2 hours 31 minutes 19 seconds,38.0 days 2.0 hours 31.0 minutes 19.0 seconds,9.09%,"[(Assessment Decision - Qualified, 3), (Put As...","[(OC - Satisfied - Other, 6.0 days 3.0 hours 3...",3292279,3.292279e+06


In [17]:
s2_t4.to_csv('engineering_AS_segment2_workflowS4_table.csv', index=False)





